In [ ]:
%reload_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
import matplotlib
matplotlib.rcParams['pdf.fonttype'] = 42 # for pdfs
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import flexiznam as flz
from cottage_analysis.analysis import spheres, common_utils
from cottage_analysis.pipelines import pipeline_utils
from v1_depth_analysis.v1_manuscript_2023 import depth_selectivity, get_session_list, depth_decoder
from v1_depth_analysis.v1_manuscript_2023 import common_utils as plt_common_utils

In [ ]:
VERSION = 6
SAVE_ROOT = Path(
    f"/camp/lab/znamenskiyp/home/shared/presentations/v1_manuscript_2023/ver{VERSION}"
)
SAVE_ROOT.mkdir(parents=True, exist_ok=True)

In [ ]:
# Load data
# Load an example session
project = "hey2_3d-vision_foodres_20220101"
session_name = "PZAH8.2h_S20230116"
flexilims_session = flz.get_flexilims_session(project)

vs_df_example, trials_df_example = spheres.sync_all_recordings(
    session_name=session_name,
    flexilims_session=flexilims_session,
    project=project,
    filter_datasets={"anatomical_only": 3},
    recording_type="two_photon",
    protocol_base="SpheresPermTubeReward",
    photodiode_protocol=5,
    return_volumes=True,
)
suite2p_ds = flz.get_datasets_recursively(
    flexilims_session=flexilims_session,
    origin_name=session_name,
    dataset_type="suite2p_traces",
)
fs = list(suite2p_ds.values())[0][-1].extra_attributes["fs"]

neurons_ds_example = pipeline_utils.create_neurons_ds(
    session_name=session_name,
    flexilims_session=flexilims_session,
    project=None,
    conflicts="skip",
)
neurons_df_example = pd.read_pickle(neurons_ds_example.path_full)

suite2p_ds = flz.get_datasets(
    flexilims_session=flexilims_session,
    origin_name=session_name,
    dataset_type="suite2p_rois",
    filter_datasets={"anatomical_only": 3},
    allow_multiple=False,
    return_dataseries=False,
)
stat = np.load(suite2p_ds.path_full / "plane0" / "stat.npy", allow_pickle=True)
iscell = np.load(suite2p_ds.path_full / "plane0" / "iscell.npy", allow_pickle=True)[
    :, 0
]
ops = np.load(suite2p_ds.path_full / "plane0" / "ops.npy", allow_pickle=True).item()

In [ ]:
# concatenate neurons_df for all sessions
# neurons_df_all = plt_common_utils.concatenate_all_neurons_df(
#     flexilims_session=flexilims_session,
#     session_list=get_session_list.get_sessions(
#         flexilims_session,
#         closedloop_only=False,
#         openloop_only=False,
#     ),
#     filename="neurons_df.pickle",
#     cols=[
#         "roi",
#         "is_depth_neuron",
#         "depth_neuron_anova_p",
#         "best_depth",
#         "preferred_depth_closedloop",
#         "depth_tuning_popt_closedloop",
#         "depth_tuning_trials_closedloop_crossval",
#         "preferred_depth_closedloop_crossval",
#         "depth_tuning_popt_closedloop_running",
#         "preferred_depth_closedloop_running",
#         "depth_tuning_popt_closedloop_notrunning",
#         "preferred_depth_closedloop_notrunning",
#         "depth_tuning_test_rsq_closedloop",
#         "depth_tuning_test_spearmanr_rval_closedloop",
#         "depth_tuning_test_spearmanr_pval_closedloop",
#     ],
#     read_iscell=True,
#     verbose=True,
# )
# neurons_df_all.to_pickle(SAVE_ROOT / "fig1/neurons_df_all.pickle")
neurons_df_all = pd.read_pickle(SAVE_ROOT / "fig1/neurons_df_all.pickle")
neurons_df_all = neurons_df_all[neurons_df_all["iscell"] == 1]

# Load the depth tuning raster
results_all = pd.read_pickle(SAVE_ROOT / "fig1/results_all.pickle")

# neurons_df_all = results_all.copy()
# neurons_df_all = neurons_df_all[neurons_df_all["iscell"] == 1]

In [ ]:
# decoder_df = plt_common_utils.concatenate_all_neurons_df(
#     flexilims_session,
#     session_list=get_session_list.get_sessions(
#         flexilims_session,
#         closedloop_only=False,
#         openloop_only=False,
#     ),
#     filename="decoder_results.pickle",
#     cols=["accuracy_closedloop", "conmat_closedloop", "best_C_closedloop", "acc_speed_bins_closedloop", "conmat_speed_bins_closedloop"],
#     read_iscell=False,
#     verbose=True,
# )

# decoder_df.to_pickle(SAVE_ROOT / "decoder_df_all.pickle")
decoder_df = pd.read_pickle(SAVE_ROOT / "decoder_df_all.pickle")

In [ ]:
# set font to arial
# plt.rcParams["font.family"] = "Arial"
fontsize_dict = {"title": 7, "label": 7, "tick": 5, "legend": 5}
cm = 1 / 2.54
# Fig.1A: Motion parallax schema & equation, VR schema
# Fig.1B: Trial structure schema, Spheres with different sizes schema

# Fig.1E: Raster plot of an example neuron
EXAMPLE_ROI = 250
fig = plt.figure(figsize=(18 * cm, 18 * cm))
depth_selectivity.plot_raster_all_depths(
    trials_df=trials_df_example,
    roi=EXAMPLE_ROI,
    is_closed_loop=True,
    corridor_length=6,
    blank_length=3,
    nbins=60,
    vmax=3,
    plot=True,
    cbar_width=0.01,
    fontsize_dict=fontsize_dict,
    position=[0.1, 0.75, 0.3, 0.15],
)

# Fig.1F: Example PSTH of a single neuron
ax = fig.add_axes([0.51, 0.81, 0.18, 0.1])
depth_selectivity.plot_PSTH(
    trials_df=trials_df_example,
    roi=EXAMPLE_ROI,
    is_closed_loop=True,
    use_col="dff",
    corridor_length=6,
    blank_length=3,
    nbins=60,
    frame_rate=fs,
    fontsize_dict=fontsize_dict,
    linewidth=1,
    legend_on=True,
    show_ci=False,
)
ax.set_xticks([])
ax.set_xlabel("")

ax = fig.add_axes([0.51, 0.75, 0.18, 0.05])
depth_selectivity.plot_PSTH(
    trials_df=trials_df_example,
    roi=EXAMPLE_ROI,
    is_closed_loop=True,
    use_col="RS",
    corridor_length=6,
    blank_length=3,
    nbins=60,
    frame_rate=fs,
    fontsize_dict=fontsize_dict,
    linewidth=1,
    legend_on=False,
    show_ci=False,
)
ax.set_ylabel("Running \nspeed (m/s)", fontsize=fontsize_dict["label"])

# Fig.1G: Example depth tuning curve & PSTH of a single neuron
ax=fig.add_axes([0.8, 0.75, 0.15, 0.15])
depth_tuning_kwargs = dict(
    rs_thr=None,
    plot_fit=True,
    plot_smooth=False,
    linewidth=2,
    linecolor="royalblue",
    closed_loop=1,
    fontsize_dict=fontsize_dict,
)
depth_selectivity.plot_running_stationary_depth_tuning(roi=EXAMPLE_ROI, roi_num=1, i=0, ax=ax, 
                                                           neurons_df=neurons_df_example,
                                                           trials_df=trials_df_example,
                                     depth_tuning_kwargs=depth_tuning_kwargs, 
                                     fontsize_dict=fontsize_dict, legend_loc="upper left")
ax.set_ylabel("\u0394F/F", fontsize=fontsize_dict["label"])
# Fig.1H: Example depth tuning curve of 6 other neuronsn (running vs not running)
EXAMPLE_ROIS = [174, 249, 319, 638, 333, 696] #[174, 249, 54, 638, 86, 88]

neurons_df_example["depth_tuned"] = neurons_df_example.apply(
    lambda x: x["depth_tuning_test_spearmanr_rval_closedloop"] > 0.1
    and x["depth_tuning_test_spearmanr_pval_closedloop"] < 0.05,
    axis=1,
)

overlay = np.zeros((ops["meanImg"].shape[0], ops["meanImg"].shape[1], 4))
for i, roi in enumerate(stat):
    if iscell[i]:
        overlay[roi["ypix"], roi["xpix"]] = [0, 1, 1, 0.25]
    # if i in EXAMPLE_ROIS or i == EXAMPLE_ROI:
    #     overlay[roi["ypix"], roi["xpix"]] = [1, 0, 0, 0.75]
    if neurons_df_example["depth_tuned"].iloc[i]:
        overlay[roi["ypix"], roi["xpix"]] = [1, 0, 0, 0.75]
        
fov_ax = fig.add_axes([0.19, 0.25, 0.48, 0.48])
depth_selectivity.plot_fov_mean_img(ops["meanImg"])
plt.imshow(np.fliplr(overlay))
fov_ax.annotate(
    "1",
    (
        ops["meanImg"].shape[0] - stat[EXAMPLE_ROI]["med"][1],
        stat[EXAMPLE_ROI]["med"][0],
    ),
    xytext=(5, 5),
    textcoords="offset points",
    color="w",
    fontsize=fontsize_dict["label"],
    arrowprops=dict(facecolor="w", edgecolor="w", arrowstyle="->"),
)
from matplotlib.lines import Line2D

legend_elements = [
    Line2D(
        [0],
        [0],
        marker="o",
        markersize=3,
        linestyle="none",
        color="cyan",
        label="All neurons",
    ),
    Line2D(
        [0],
        [0],
        marker="o",
        markersize=3,
        linestyle="none",
        color="red",
        label="Depth-tuned neurons",
    ),
]
fov_ax.legend(
    handles=legend_elements,
    loc="upper right",
    fontsize=fontsize_dict["label"],
    framealpha=1,
    borderpad=0.6,
)

for i, roi in enumerate(EXAMPLE_ROIS):
    ax = fig.add_axes([i // 3 * 0.56+0.06, 0.57 - i % 3 * 0.13, 0.11, 0.1])
    if i<3:
        label_pos="upper_right"
    else:
        label_pos="upper_left"
    
    roi_num = i+2
    depth_selectivity.plot_running_stationary_depth_tuning(roi=roi, roi_num=roi_num, i=i, 
                                                           neurons_df=neurons_df_example,
                                                           trials_df=trials_df_example,
                                                           ax=ax, fov_ax=fov_ax, ops=ops, stat=stat,
                                        depth_tuning_kwargs=depth_tuning_kwargs, 
                                        fontsize_dict=fontsize_dict, legend_loc="upper right", text_pos=label_pos)
    
# Fig.1G: Histogram of proportion of depth-tuned neurons;
# preferred depths of all depth-tuned neurons;
# Raster plot of preferred depths of all neurons (sorted by preferred depth)
neurons_df_all["depth_tuned"] = neurons_df_all.apply(
    lambda x: x["depth_tuning_test_spearmanr_rval_closedloop"] > 0.1
    and x["depth_tuning_test_spearmanr_pval_closedloop"] < 0.05,
    axis=1,
)

fig.add_axes([0.8, 0.54, 0.16, 0.13])
depth_selectivity.plot_depth_neuron_perc_hist(
    results_df=neurons_df_all,
    bins=np.arange(0, 1.0, 0.1),
    fontsize_dict=fontsize_dict,
)


fig.add_axes([0.8, 0.31, 0.16, 0.13])
depth_selectivity.plot_preferred_depth_hist(
    results_df=neurons_df_all[(neurons_df_all["depth_tuned"] == 1)],
    use_col="preferred_depth_closedloop_crossval",
    nbins=20,
    fontsize_dict=fontsize_dict,
)

results_all["ndepths"] = results_all["psth_crossval"].apply(lambda x: len(x) - 1)
neurons_resp = results_all[
    (results_all["ndepths"] == 8)
    & (results_all["depth_tuning_test_spearmanr_rval_closedloop"] > 0.1)
    & (results_all["depth_tuning_test_spearmanr_pval_closedloop"] < 0.05)
    & (results_all["iscell"] == 1)
]
fig.add_axes([0.055, 0.08, 0.45, 0.15])
depth_selectivity.plot_psth_raster(
    results_df=neurons_resp,
    depth_list=np.geomspace(5, 640, num=8).astype(int),
    fontsize_dict=fontsize_dict,
)

# Plot decoder accuracy - running speed bins
fig.add_axes([0.65, 0.08, 0.3, 0.15])
decoder_df = decoder_df[decoder_df["acc_speed_bins_closedloop"].notnull()]
decoder_df = depth_decoder.calculate_error_all_sessions(decoder_df)
_ = depth_decoder.plot_decoder_err_by_speeds(decoder_df, 
                           np.linspace(0.1, 1.5, 8), 
                           closed_loop=1, 
                           linecolor="k", 
                           linewidth=1, 
                           alpha=1, 
                           markersize=5, 
                           fontsize_dict=fontsize_dict)

fig.savefig(SAVE_ROOT / "fig1.pdf", dpi=300)
fig.savefig(SAVE_ROOT / "fig1.png", dpi=300)

In [ ]:
def get_bootstrap_ci(arr, sig_level=0.05, n_bootstraps=1000, func=np.nanmean):
    """Calculate confidence interval using bootstrap method.

    Args:
        arr (np.ndarray): 1d or 2d array, for example ndepths x ntrials to calculate confidence interval across trials.
        sig_level (float): Significant level.
        n_bootstraps (int): Number of bootstraps.

    Returns:
        CI_low (np.1darray): lower bound of confidence interval.
        CI_high (np.1darray): upper bound of confidence interval.
    """
    if arr.ndim == 1:
        arr = arr[np.newaxis, :]
    ndepths, ntrials = arr.shape
    bootstrapped_means = np.zeros((n_bootstraps, ndepths))
    for i in range(n_bootstraps):
        for idepth in range(ndepths):
            resampled = arr[idepth, np.random.randint(0, ntrials, ntrials)]
            bootstrapped_means[i, idepth] = func(resampled)
    CI_low = np.percentile(bootstrapped_means, 100 * sig_level / 2, axis=0)
    CI_high = np.percentile(bootstrapped_means, 100 * (1 - sig_level / 2), axis=0)
    return CI_low, CI_high, bootstrapped_means

from scipy.stats import ttest_1samp
all_speed_bin_edges=np.linspace(0.1, 1.1, 6)
decoder_df_5depths = decoder_df[decoder_df['session'].str.contains(f'PZAH6.4b|PZAG3.4f', regex=True)]   
decoder_df_8depths = decoder_df[decoder_df['session'].str.contains(f'PZAH8.2|PZAH10.2', regex=True)] 
use_nbins=len(all_speed_bin_edges)+1
speed_bins = all_speed_bin_edges[:use_nbins]  
for i, (decoder_df_sub, label, ndepths) in enumerate(zip([decoder_df_5depths, decoder_df_8depths],
                                                ["5 depths", "8 depths"],
                                                [5,8])):
    # cut every trial to the same length
    decoder_df_sub["acc_speed_bins_closedloop_cut"] = decoder_df_sub.apply(lambda x: x.acc_speed_bins_closedloop[:use_nbins], axis=1)
    acc_speed_bins = np.stack(decoder_df_sub.acc_speed_bins_closedloop_cut)
    print(ttest_1samp(acc_speed_bins[:,0], 1/ndepths))
    _, _, bootstrapped_means = get_bootstrap_ci(acc_speed_bins[:,0])
    print(np.nanmean(bootstrapped_means<1/ndepths))

In [ ]:
# CHECK FALSE POSITIVES ARE 5 % (SHUFFLED DISTRIBUTION FOR A FEW SESSIONS)
# put 5 depths raster in the supp
# CAD plot of stimuli (softwares that can draw 3d stuff, photoshop)
# MINOR: histogram: add dash line indicating the depth range in the experiment

## SUPP


In [ ]:
neurons_resp = results_all[
    (results_all["ndepths"] == 8)
    & (results_all["depth_tuning_test_spearmanr_rval_closedloop"] > 0.1)
    & (results_all["iscell"] == 1)
]
plt.figure()
depth_selectivity.plot_psth_raster(
    results_df=neurons_resp,
    depth_list=np.geomspace(5, 640, num=8).astype(int),
    fontsize_dict=fontsize_dict,
)

neurons_nonresp = results_all[
    (results_all["ndepths"] == 8)
    & (results_all["iscell"] == 1)
    & (results_all["depth_tuning_test_spearmanr_rval_closedloop"] < 0.1)
]
plt.figure()
depth_selectivity.plot_psth_raster(
    results_df=neurons_nonresp,
    depth_list=np.geomspace(5, 640, num=8).astype(int),
    fontsize_dict=fontsize_dict,
)

In [ ]:
select_neurons = ((neurons_df_all["depth_tuning_test_spearmanr_rval_closedloop"] > 0.1)
    & (neurons_df_all["depth_tuning_test_spearmanr_pval_closedloop"] < 0.05)
    # & (neurons_df_all['depth_tuning_test_rsq_closedloop'] > 0.02)
    & ~(neurons_df_all['session'].str.contains("PZAG3.4f"))
)


running_amp = np.exp(np.stack(neurons_df_all[select_neurons]['depth_tuning_popt_closedloop_running'])[:,0]) + np.stack(neurons_df_all[select_neurons]['depth_tuning_popt_closedloop_running'])[:,-1]
notrunning_amp = np.exp(np.stack(neurons_df_all[select_neurons]['depth_tuning_popt_closedloop_notrunning'])[:,0]) + np.stack(neurons_df_all[select_neurons]['depth_tuning_popt_closedloop_notrunning'])[:,-1]


plt.scatter(running_amp, notrunning_amp, s=1, alpha=0.05)
# plt.xscale("log")
# plt.yscale("log")
plt.gca().set_aspect('equal', adjustable='box')
plt.xlim([0,1])
plt.ylim([0,1])
plt.plot([1e-2, 1e2], [1e-2, 1e2], 'r--')
plt.xlabel("running")
plt.ylabel("stationary")

In [ ]:
from matplotlib_venn import venn3

depth_neurons = neurons_df_6f[
    (neurons_df_6f["depth_tuning_test_spearmanr_pval_closedloop"] < 0.05)
    & (neurons_df_6f["iscell"] == 1)
].index.values
visually_responsive = neurons_df_6f[
    (neurons_df_6f["visually_responsive"] == 1) & (neurons_df_6f["iscell"] == 1)
].index.values
all_neurons = neurons_df_6f[neurons_df_6f["iscell"] == 1].index.values
venn3(
    [set(depth_neurons), set(visually_responsive), set(all_neurons)],
    ("Depth-tuned", "Visually responsive", "All neurons"),
)
print(
    len(visually_responsive) / len(all_neurons), len(depth_neurons) / len(all_neurons)
)

In [ ]:
fig = plt.figure()
# Fig.1F: Example depth tuning curve & PSTH of 3 other neurons
EXAMPLE_ROIS = [249, 54, 88]
for i, roi in enumerate(EXAMPLE_ROIS):
    if i == len(EXAMPLE_ROIS) - 1:
        legend_on = True
    else:
        legend_on = False
    depth_selectivity.plot_PSTH(
        fig=fig,
        trials_df=trials_df_example,
        roi=roi,
        is_closed_loop=True,
        max_distance=6,
        nbins=20,
        frame_rate=15,
        plot_x=0.05 + 0.25 * (i + 1),
        plot_y=0.67,
        plot_width=0.2,
        plot_height=0.18,
        fontsize_dict={"title": 15, "label": 10, "tick": 10, "legend": 5},
        linewidth=2,
        legend_on=legend_on,
    )
    if i != 0:
        plt.ylabel("")

    ax2 = depth_selectivity.plot_depth_tuning_curve(
        fig=fig,
        neurons_df=neurons_df_example,
        trials_df=trials_df_example,
        roi=roi,
        rs_thr=0.2,
        plot_fit=False,
        linewidth=2,
        linecolor="r",
        fit_linecolor="r",
        closed_loop=1,
        plot_x=0.05 + 0.25 * (i + 1) + 0.12,
        plot_y=0.79,
        plot_width=0.08,
        plot_height=0.08,
        fontsize_dict={"title": 15, "label": 10, "tick": 10},
    )
    plt.ylabel("")
    plt.xlabel("")
    plt.xticks([])

    for rs_thr, rs_thr_max, still_only, still_time, label, linecolor in zip(
        [None, 0.05, None],
        [None, 0.2, 0.05],
        [False, False, True],
        [0, 0, 1],
        ["All", "Walking\n(5-20cm/s)", "Not running\nfor 1s before"],
        ["b", "orange", "gray"],
    ):
        depth_selectivity.plot_depth_tuning_curve(
            fig=fig,
            neurons_df=neurons_df_example,
            trials_df=trials_df_example,
            roi=roi,
            rs_thr=rs_thr,
            rs_thr_max=rs_thr_max,
            still_only=still_only,
            still_time=still_time,
            plot_fit=False,
            linewidth=2,
            linecolor=linecolor,
            fit_linecolor=None,
            closed_loop=1,
            overwrite_ax=False,
            ax=ax2,
            plot_x=0.05 + 0.25 * (i + 1) + 0.12,
            plot_y=0.79,
            plot_width=0.08,
            plot_height=0.08,
            label=label,
            fontsize_dict={"title": 15, "label": 10, "tick": 10},
        )
        plt.ylabel("")
        plt.xlabel("")
        plt.xticks([])
    if i == len(EXAMPLE_ROIS) - 1:
        plt.legend(
            frameon=False, handlelength=1, bbox_to_anchor=[0.51, 0.51], fontsize=5
        )

In [ ]:
# Depth tuning curve & PSTH of different running states
fig = plt.figure()
rois = [250, 249, 54, 88]
for i, roi in enumerate(rois):
    # depth tuning curve
    ax = depth_selectivity.plot_depth_tuning_curve(
        fig=fig,
        neurons_df=neurons_df_example,
        trials_df=trials_df_example,
        roi=roi,
        rs_thr=0.2,
        plot_fit=False,
        linewidth=2,
        linecolor="r",
        fit_linecolor="r",
        closed_loop=1,
        plot_x=0 + 0.25 * (i + 1),
        plot_y=1,
        plot_width=0.2,
        plot_height=0.2,
        label="Running (> 20cm/s)",
        fontsize_dict={"title": 15, "label": 10, "tick": 10},
    )
    if i != 0:
        plt.ylabel("")

    depth_selectivity.plot_depth_tuning_curve(
        fig=fig,
        neurons_df=neurons_df_example,
        trials_df=trials_df_example,
        roi=roi,
        rs_thr=None,
        rs_thr_max=None,
        still_only=False,
        plot_fit=False,
        linewidth=2,
        linecolor="b",
        fit_linecolor="r",
        closed_loop=1,
        overwrite_ax=False,
        ax=ax,
        plot_x=0 + 0.25 * (i + 1),
        plot_y=1,
        plot_width=0.2,
        plot_height=0.2,
        label="All",
        fontsize_dict={"title": 15, "label": 10, "tick": 10},
    )
    if i != 0:
        plt.ylabel("")

    depth_selectivity.plot_depth_tuning_curve(
        fig=fig,
        neurons_df=neurons_df_example,
        trials_df=trials_df_example,
        roi=roi,
        rs_thr=0.05,
        rs_thr_max=0.2,
        still_only=False,
        plot_fit=False,
        linewidth=2,
        linecolor="orange",
        fit_linecolor="r",
        closed_loop=1,
        overwrite_ax=False,
        ax=ax,
        plot_x=0 + 0.25 * (i + 1),
        plot_y=1,
        plot_width=0.2,
        plot_height=0.2,
        label="Walking (5-20cm/s)",
        fontsize_dict={"title": 15, "label": 10, "tick": 10},
    )
    if i != 0:
        plt.ylabel("")

    depth_selectivity.plot_depth_tuning_curve(
        fig=fig,
        neurons_df=neurons_df_example,
        trials_df=trials_df_example,
        roi=roi,
        rs_thr=None,
        rs_thr_max=0.05,
        still_only=True,
        still_time=1,
        frame_rate=15,
        plot_fit=False,
        linewidth=2,
        linecolor="gray",
        fit_linecolor="r",
        closed_loop=1,
        overwrite_ax=False,
        ax=ax,
        plot_x=0 + 0.15 * (i + 1),
        plot_y=1,
        plot_width=0.2,
        plot_height=0.2,
        label="Not running for 1s before",
        fontsize_dict={"title": 15, "label": 10, "tick": 10},
    )
    if i != 0:
        plt.ylabel("")
    if i == len(rois) - 1:
        plt.legend(fontsize=5)

    # PSTH
    depth_selectivity.plot_PSTH(
        fig=fig,
        trials_df=trials_df_example,
        roi=roi,
        is_closed_loop=True,
        max_distance=6,
        nbins=20,
        frame_rate=15,
        rs_thr_min=0.2,
        still_only=False,
        still_time=0,
        plot_x=0 + 0.25 * (i + 1),
        plot_y=0.47,
        plot_width=0.15,
        plot_height=0.1,
        fontsize_dict={"title": 15, "label": 10, "tick": 10, "legend": 5},
        linewidth=2,
        legend_on=False,
    )
    if i != 0:
        plt.ylabel("")
    if i == 0:
        plt.title("Running (> 20cm/s)")
    ylim = plt.gca().get_ylim()
    plt.xlabel("")

    depth_selectivity.plot_PSTH(
        fig=fig,
        trials_df=trials_df_example,
        roi=roi,
        is_closed_loop=True,
        max_distance=6,
        nbins=20,
        frame_rate=15,
        rs_thr_min=None,
        still_only=False,
        still_time=0,
        plot_x=0 + 0.25 * (i + 1),
        plot_y=0.7,
        plot_width=0.15,
        plot_height=0.1,
        fontsize_dict={"title": 15, "label": 10, "tick": 10, "legend": 5},
        linewidth=2,
        legend_on=False,
    )
    if i != 0:
        plt.ylabel("")
    if i == 0:
        plt.title("All frames")
    plt.ylim(ylim)
    plt.xlabel("")

    depth_selectivity.plot_PSTH(
        fig=fig,
        trials_df=trials_df_example,
        roi=roi,
        is_closed_loop=True,
        max_distance=6,
        nbins=20,
        frame_rate=15,
        rs_thr_min=0.05,
        rs_thr_max=0.2,
        still_only=False,
        still_time=0,
        plot_x=0 + 0.25 * (i + 1),
        plot_y=0.24,
        plot_width=0.15,
        plot_height=0.1,
        fontsize_dict={"title": 15, "label": 10, "tick": 10, "legend": 5},
        linewidth=2,
        legend_on=False,
    )
    if i != 0:
        plt.ylabel("")
    if i == 0:
        plt.title("Walking (5-20cm/s)")
    plt.ylim(ylim)
    plt.xlabel("")

    depth_selectivity.plot_PSTH(
        fig=fig,
        trials_df=trials_df_example,
        roi=roi,
        is_closed_loop=True,
        max_distance=6,
        nbins=20,
        frame_rate=15,
        rs_thr_min=None,
        rs_thr_max=0.05,
        still_only=True,
        still_time=1,
        plot_x=0 + 0.25 * (i + 1),
        plot_y=0,
        plot_width=0.15,
        plot_height=0.1,
        fontsize_dict={"title": 15, "label": 10, "tick": 10, "legend": 5},
        linewidth=2,
        legend_on=False,
    )
    if i != 0:
        plt.ylabel("")
    if i == 0:
        plt.title("Not running for 1s before")
    plt.ylim(ylim)

# QUANTIFY: select depth-tuned neurons based on all frames, compare max response from running / non-running